<a href="https://colab.research.google.com/github/emilstahl97/Scalable-Machine-Learning-and-Deep-Learning-ID2223/blob/dev/merged_lab2_id2223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Scalable Machine Learning and Deep Learning - Lab 2**

By:

Erik Kongpachith,
Emil Ståhl, 
Selemawit Fsha Nguse

1. S-BERT Regression Objective
2. S-BERT Classificatiob Objective
3. Semantic Search System




Add dependencies

In [1]:
!pip install sentence_transformers
!pip install transformers
!pip install tokenizers
!pip install torch
!pip install wget
!pip install pyspark

     |████████████████████████████████| 78 kB 1.6 MB/s 
     |████████████████████████████████| 3.4 MB 6.0 MB/s 
     |████████████████████████████████| 3.3 MB 14.0 MB/s 
     |████████████████████████████████| 1.2 MB 32.4 MB/s 
     |████████████████████████████████| 61 kB 416 kB/s 
     |████████████████████████████████| 596 kB 31.8 MB/s 
     |████████████████████████████████| 895 kB 33.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=2c72a46033824269e624c3c1528b7fffa10478fff5f0673b5a8774ea54a5fbdb
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=db7d922c1c5d7b7b544a0734ac1b7c526ce8b348f0a

Import libraries

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer


import os
import re
import csv
import wget
import json
import math
import scipy
import torch
import string
import sklearn

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input

from sentence_transformers import SentenceTransformer
from sentence_transformers import LoggingHandler
from sentence_transformers import models, losses, util
from sentence_transformers.readers import InputExample
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

from transformers import BertTokenizer, TFBertModel, BertConfig
from tokenizers import BertWordPieceTokenizer

from torch.utils.data import DataLoader

from datetime import datetime

**Mount Google Drive to load saved models**

In [3]:
# README - Execute this cell to mount the notebook in your google drive. 
# Execute the cell and follow the link to sign and, paste the given key in the little text box. The credentials are only available for you. 

import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

if not os.path.exists("/content/drive/MyDrive/stsbenchmark"): 
  os.mkdir("/content/drive/MyDrive/stsbenchmark")
  os.chdir("/content/drive/MyDrive/stsbenchmark")
  !git clone https://github.com/emilstahl97/stsbenchmark.git
  !git pull
else:
  print("Dataset exists")
  os.chdir("/content/drive/MyDrive/stsbenchmark")

train_path = "/content/drive/MyDrive/stsbenchmark/stsbenchmark/sts-train.csv"
test_path = "/content/drive/MyDrive/stsbenchmark/stsbenchmark/sts-test.csv"
dev_path = "/content/drive/MyDrive/stsbenchmark/stsbenchmark/sts-dev.csv"

news_path = "/content/drive/MyDrive/stsbenchmark/stsbenchmark/news.csv"

# saved models
if not os.path.exists("/content/drive/MyDrive/id2223/lab2/models"):
  os.makedirs("/content/drive/MyDrive/id2223/lab2/models")

if not os.path.exists("/content/drive/MyDrive/id2223/lab2/results"):
  os.makedirs("/content/drive/MyDrive/id2223/lab2/results/regression_results/")


regression_model_path = "/content/drive/MyDrive/id2223/lab2/models/regression_model"
regression_results_path = "/content/drive/MyDrive/id2223/lab2/models/regression_results"
classification_model_path = "/content/drive/MyDrive/id2223/lab2/models/classification_model"


Mounted at /content/drive
Dataset exists


## **S-BERT REGRESSION OBJECTIVE**

In [4]:
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [5]:
train_samples = []
test_samples = []
dev_samples = []

columns = ['title', 'type', 'year', 'id', 'score', 'sentence_1', 'sentence_2']
paths = {"train": train_path, "test": test_path, "dev": dev_path}

for key, path in paths.items():
  with open(path, newline='') as f:
    temp = csv.DictReader(f, delimiter='\t', fieldnames=columns, quoting=csv.QUOTE_NONE)
    for row in temp:
        score = float(row['score']) / 2.5 - 1 
        input_example = InputExample(texts=[row['sentence_1'], row['sentence_2']], label=score)
        if (key == "train"):
          train_samples.append(input_example)
        elif key == "test":
          test_samples.append(input_example)
        elif key == "dev":
          dev_samples.append(input_example)
        else:
          raise Exception(f"key {key} not known, exiting")

In [6]:
train_batch_size = 16
learn_rate = 2e-5
num_epochs = 1

Mean-pooling strategy

In [7]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

Define the model

In [8]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Load the training set and define the loss function as the cosine similarity

In [9]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

Define the evaluator for the sentence embeddings

In [10]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=train_batch_size, name='sts-dev')

10% of train dataset for warm-up

In [11]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

**Training**

In [12]:
if os.path.exists(regression_model_path):
    print("Loading pre-trained model")
    model = SentenceTransformer(regression_model_path)
else:
    print("Re-training model")
    model = model.fit(train_objectives=[(train_dataloader, train_loss)],
                    optimizer_class=torch.optim.Adam,
                    optimizer_params={'lr': learn_rate},
                    evaluator=evaluator,
                    epochs=num_epochs,
                    evaluation_steps=1000,
                    warmup_steps=warmup_steps,
                    output_path=regression_model_path)

Loading pre-trained model


**Evaluation on STS benchmark dataset**

Mathematical relationship: *cosine_similarity = 1 - cosine_distance*

In [13]:
test_eval = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=train_batch_size, name='sts-test')
c_s = test_eval(model, output_path=regression_results_path)
print('Cosine similarity = ', c_s)

Cosine similarity =  0.5263565694181714


Embedding sentences

In [14]:
df_test = pd.read_csv(test_path, sep='\t', header=None, error_bad_lines=False, quoting=csv.QUOTE_NONE)
df_test.columns = columns

b'Skipping line 626: expected 7 fields, saw 9\nSkipping line 627: expected 7 fields, saw 9\nSkipping line 628: expected 7 fields, saw 9\nSkipping line 629: expected 7 fields, saw 9\nSkipping line 630: expected 7 fields, saw 9\nSkipping line 631: expected 7 fields, saw 9\nSkipping line 632: expected 7 fields, saw 9\nSkipping line 633: expected 7 fields, saw 9\nSkipping line 634: expected 7 fields, saw 9\nSkipping line 635: expected 7 fields, saw 9\nSkipping line 636: expected 7 fields, saw 9\nSkipping line 637: expected 7 fields, saw 9\nSkipping line 638: expected 7 fields, saw 9\nSkipping line 639: expected 7 fields, saw 9\nSkipping line 640: expected 7 fields, saw 9\nSkipping line 641: expected 7 fields, saw 9\nSkipping line 642: expected 7 fields, saw 9\nSkipping line 643: expected 7 fields, saw 9\nSkipping line 644: expected 7 fields, saw 9\nSkipping line 645: expected 7 fields, saw 9\nSkipping line 646: expected 7 fields, saw 9\nSkipping line 647: expected 7 fields, saw 9\nSkipping

In [15]:
embed_1 = model.encode(df_test['sentence_1'], convert_to_numpy=True, batch_size=train_batch_size)
embed_2 = model.encode(df_test['sentence_2'], convert_to_numpy=True, batch_size=train_batch_size)

Compute the cosine similarity

In [16]:
cos_sim = 1 - sklearn.metrics.pairwise.paired_cosine_distances(embed_1, embed_2)
print('Cosine similarity = ', cos_sim)

Cosine similarity =  [0.27117753 0.8809062  0.58966494 ... 0.74274147 0.85241306 0.9187385 ]


Spearmean correlation coefficient

In [17]:
spr_corr = scipy.stats.spearmanr(cos_sim, df_test['score'])
print('Spearmean correlation coefficient = ', spr_corr[0])

Spearmean correlation coefficient =  0.534628805412517


## *2. S-BERT CLASSIFICATION OBJECTIVE*

In [18]:
print('Downloading snil: ')
wget.download('https://nlp.stanford.edu/projects/snli/snli_1.0.zip', './snli_1.0.zip')
!unzip snli_1.0.zip

Archive:  snli_1.0.zip
replace snli_1.0/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: snli_1.0/.DS_Store      
  inflating: __MACOSX/snli_1.0/._.DS_Store  
 extracting: snli_1.0/Icon           
  inflating: __MACOSX/snli_1.0/._Icon  
  inflating: snli_1.0/README.txt     
  inflating: __MACOSX/snli_1.0/._README.txt  
  inflating: snli_1.0/snli_1.0_dev.jsonl  
  inflating: snli_1.0/snli_1.0_dev.txt  
  inflating: snli_1.0/snli_1.0_test.jsonl  
  inflating: snli_1.0/snli_1.0_test.txt  
  inflating: snli_1.0/snli_1.0_train.jsonl  
  inflating: snli_1.0/snli_1.0_train.txt  
  inflating: __MACOSX/._snli_1.0     


In [19]:
spark = SparkSession.builder.getOrCreate()
train_class_path = 'snli_1.0/snli_1.0_train.jsonl'
train_class = spark.read.json(train_class_path)

test_class_path = 'snli_1.0/snli_1.0_test.jsonl'
test_class = spark.read.json(test_class_path)

dev_class_path = 'snli_1.0/snli_1.0_dev.jsonl'
dev_class = spark.read.json(dev_class_path)

In [20]:
indexer = StringIndexer(inputCol="gold_label", outputCol="label")
def CreateSamples(df):
    samples = []
    for _, row in df.iterrows():
        input_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['label'])
        samples.append(input_example)
    return samples
    
def CreateSampleClass(df):
    df = df.filter(col("gold_label") != "-")
    df = indexer.fit(df).transform(df)
    df = df.withColumn("label", col("label").cast('int'))

    df_class = df.select("sentence1", "sentence2", "label").toPandas()

    return CreateSamples(df_class)

In [21]:
train_class_samples = CreateSampleClass(train_class)
test_class_samples = CreateSampleClass(test_class)
dev_class_samples = CreateSampleClass(dev_class)

In [22]:
train_dataloader_cl = DataLoader(train_class_samples, shuffle=True, batch_size=train_batch_size)
num_lables = test_class.select('annotator_labels').distinct().count()
train_loss_cl = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=num_lables)

In [23]:
evaluator_cl = EmbeddingSimilarityEvaluator.from_input_examples(dev_class_samples, batch_size=train_batch_size, name='snli-dev')
warmup_steps_cl = math.ceil(len(train_dataloader) * num_epochs * 0.1)

In [24]:
model_class_location = '/content/drive/MyDrive/stsbenchmark/stsbenchmark/models/training_snli'

if os.path.exists(model_class_location):
    print("Loading pre-trained model")
    model = SentenceTransformer(model_class_location)
else:
    print("Retraining model")
    model.fit(train_objectives=[(train_dataloader_cl, train_loss_cl)],
             evaluator=evaluator_cl,
             epochs=num_epochs,
             evaluation_steps=1000,
             warmup_steps=warmup_steps_cl,
             output_path=model_class_location)

Loading pre-trained model


In [25]:
evaluation_class_location = "./classification"

if not os.path.exists(evaluation_class_location):
    os.makedirs(evaluation_class_location)

test_eval_cl = EmbeddingSimilarityEvaluator.from_input_examples(test_class_samples, batch_size=train_batch_size, name='snli-test')
c_s_cl = test_eval_cl(model, output_path=evaluation_class_location)
print('Cosine similarity with the sentence_transformers library = ', c_s_cl)

Cosine similarity with the sentence_transformers library =  -0.10038784683009352


In [ ]:
c_s_sts = test_eval(model, output_path=evaluation_class_location)
print('Cosine similarity with the sentence_transformers library = ', c_s_sts)

In [ ]:
embed_1_snli = model.encode(df_test['sentence_1'], convert_to_numpy=True, batch_size=train_batch_size)
embed_2_snli = model.encode(df_test['sentence_2'], convert_to_numpy=True, batch_size=train_batch_size)

embed_1 = model.encode(df_test['sentence_1'], convert_to_numpy=True, batch_size=train_batch_size)
embed_2 = model.encode(df_test['sentence_2'], convert_to_numpy=True, batch_size=train_batch_size)

In [ ]:
cos_sim_cl = 1 - sklearn.metrics.pairwise.paired_cosine_distances(embed_1_snli, embed_2_snli)
print('SNLI-test: cosine similarity = ', cos_sim_cl)

cos_sim_sts = 1 - sklearn.metrics.pairwise.paired_cosine_distances(embed_1, embed_2)
print('STS benchmark: cosine similarity = ', cos_sim_sts)

In [ ]:
spr_corr_cl = scipy.stats.spearmanr(cos_sim_cl, df_test['score'])
print('SNLI-test: Spearmean correlation coefficient = ', spr_corr_cl[0])

spr_corr_sts = scipy.stats.spearmanr(cos_sim_sts, df_test['score'])
print('STS benchmark: Spearmean correlation coefficient = ', spr_corr_sts[0])

##**3. Semantic Search**

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
# Import the data, remove the first row ie the header and dates
df = pd.read_csv(news_path, sep =",", error_bad_lines=False, names = ["Date", "Sentence"], nrows=10000)
df = df.iloc[1: , :]
df.drop(['Date'], axis=1, inplace = True)
df.head(10)

In [ ]:
# Compute the cosine similarity between the search sentences and the sentences from data, return the indexes of the k most similar sentences
def search(search_sentences_embedding, sentence_embeddings):
    similarity_matrix = cosine_similarity(search_sentences_embedding, sentence_embeddings[:])
    similarities = similarity_matrix[0]
    return similarities.argsort()[-k:][::1]

In [ ]:
sentences = df.iloc[:, 0] # Get the sentences from the data

sentence_embeddings= model.encode(sentences.values) # do embedding for the sentences from the data

In [ ]:
search_sentence_embedding = model.encode(["kallis out of bangladesh"]) #Embedd the search sentence

k = 10 # number of similar sentences to include in the result
similar_sentences = search(search_sentence_embedding, sentence_embeddings) #get indexes for the most similar sentences

In [ ]:
# Print out the K most similar sentences
for idx in similar_sentences:
    print(sentences[idx])